In [1]:
from aux_func_V3 import *
import API_Numpy
import API_TensorFlow

import dill
import seaborn
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [2]:
def MalhaRetangular(N):
   Δ = 10.0/N
   Δx = Δ; Δy = Δ
   x = API_Numpy.arange(-5.0,5.0+Δx,Δx)
   y = API_Numpy.arange(-1.0,1.0,Δy)

   X = API_Numpy.stack([x]*y.shape[0],axis=1)
   Y = API_Numpy.stack([y]*x.shape[0],axis=0)

   return X, Y, Δ

def CondiçãoInicialShockEntropy2D(X, Y, γ,θ):
   R = API_Numpy.zeros(X.shape)
   P = API_Numpy.ones(X.shape)
   U = API_Numpy.zeros(X.shape)
   V = API_Numpy.zeros(X.shape)

   I = API_Numpy.ones(X.shape)

   l=X < -4
   r=X >=-4

   R[l]=I[l]*27.0/7.0
   U[l]=I[l]*4.0*API_Numpy.sqrt(35.0)/9.0
   P[l]=I[l]*31.0/3.0

   R[r] = 1.0 + API_Numpy.sin(X[r]*API_Numpy.cos(θ)*2.0*API_Numpy.pi + Y[r]*API_Numpy.sin(θ)*2.0*API_Numpy.pi)/5.0

   E = P/(γ-1.0) + R*(U**2 + V**2)/2.0
   Q0 = API_Numpy.stack([R, R*U, R*V, E],axis=0)
   return Q0

def CondiçãoInicialShockEntropy2D_N(N, γ,θ):
   X, Y, Δ = MalhaRetangular(N)
   Q0 = CondiçãoInicialShockEntropy2D(X, Y, γ,θ)
   return X, Y, Δ, Q0

def NullForce(Q,API):
   return 0.0

In [3]:
class ShockEntropy2DGhostPointsX:
    def __init__(self,γ,Δx,y,θ):
        self.γ=γ
        self.Δx=Δx
        self.y=y
        self.θ=θ
    def __call__(self,Q,API,t=None):
        ρl = 27.0/7.0
        ul = 4.0*API.cast(API.sqrt(35.0),'float64')/9.0
        pl = 31.0/3.0
        El = pl/(γ-1.0) + ρl*(ul**2)/2.0
        El = API.cast(El,'float64')

        raw_ref_shape=API.shape(Q)[:-3]
        out_shape=API.concat([API.shape(Q)[:-2],[3],API.shape(Q)[-1:]],axis=0)
        ref_shape=API.concat([API.ones(API.shape(raw_ref_shape),dtype='int32'),[4],[1],[1]],axis=0)

        vals_left=API.reshape([ρl, ρl*ul, 0.0, El],ref_shape)

        Xr = API.reshape([5.0+self.Δx, 5.0+2*self.Δx, 5.0+3*self.Δx], [3, 1])
        Yr = self.y[...,:3,:]
        ρr = 1.0 + API.sin(5.0*Xr*API.cos(self.θ) + 5.0*Yr*API.sin(self.θ))/5.0
        pr = API.ones(ρr.shape)
        ur = API.zeros(ρr.shape)
        vr = API.zeros(ρr.shape)
        Er = pr/(self.γ-1.0)
        Qr = API.stack([ρr, ur, vr, Er],axis=-3)

        Qg = [API.ones(out_shape,'float64')*API.cast(vals_left,'float64'),
        Q,
        API.cast(Qr,'float64')]

        return API.concat(Qg,-2)

class ShockEntropy2DGhostPointsY:
    def __init__(self):
        pass
    def __call__(self,Q,API,t=None):
        Qg = API.concat(
            [Q[...,:,-3:],
                Q,
                Q[...,:,:3]],axis=-1)
        return Qg

In [4]:
γ = 7.0/5.0
θ = API_Numpy.pi/6.0
N = 400
x, y, Δ, U0 = CondiçãoInicialShockEntropy2D_N(N, γ,θ)
Δx=Δ
cfl = 0.5
t_final = 0.01

GhostPointsX=ShockEntropy2DGhostPointsX(γ,Δx,y,θ)
GhostPointsY=ShockEntropy2DGhostPointsY()

In [6]:
API=API_TensorFlow
# API=API_Numpy

ε=1e-40

WENO_Z_sim=simulation_2D(API,
                        euler_equation_2D,
                        WENO_Z,
                        γ=γ)

WENO_Zp_sim=simulation_2D(API,
                        euler_equation_2D,
                        WENO_Zp,
                        γ=γ)

WENO_Z_MS_sim=simulation_2D(API,
                        euler_equation_2D,
                        WENO_Z,
                        map_function=Hong_mapping,
                        mapping=pre_mapping,
                        γ=γ)

WENO_Zp_MS_sim=simulation_2D(API,
                        euler_equation_2D,
                        WENO_Zp,
                        map_function=Hong_mapping,
                        mapping=pre_mapping,
                        γ=γ)
                        
WENO_Zp_BI_sim=simulation_2D(API,
                        euler_equation_2D,
                        WENO_Zp,
                        map_function=BI_mapping,
                        mapping=pre_mapping,
                        γ=γ)

WENO_Z_BI_sim=simulation_2D(API,
                        euler_equation_2D,
                        WENO_Z,
                        map_function=BI_mapping,
                        mapping=pre_mapping,
                        γ=γ)

WENO_JS_BI_sim=simulation_2D(API,
                        euler_equation_2D,
                        WENO_JS,
                        map_function=BI_mapping,
                        mapping=pre_mapping,
                        γ=γ)

WENO_ZC_MS_sim=simulation_2D(API,
                        euler_equation_2D,
                        WENO_ZC,
                        map_function=Hong_mapping,
                        mapping=pre_inv_mapping,
                        γ=γ)

WENO_ZC_BI_sim=simulation_2D(API,
                        euler_equation_2D,
                        WENO_ZC,
                        map_function=BI_mapping,
                        mapping=pre_inv_mapping,
                        γ=γ)

In [7]:
import gif
@gif.frame
def helper_gif(ref_U,x,y):
    plt.figure(figsize=(8,2*8))
    a=plt.pcolormesh(x,y,ref_U,cmap='jet',vmin=0.75,vmax=4.75)
    plt.colorbar(a)
    plt.contour(x,y,ref_U,vmin=1,vmax=23,levels=15,colors='black',linewidths=0.5)
    plt.title(name)

def helper_plot(ref_U,x,y):
    plt.figure(figsize=(8,2*8))
    a=plt.pcolormesh(x,y,ref_U,cmap='jet'vmin=0.75,vmax=4.75)
    plt.colorbar(a)
    plt.contour(x,y,ref_U,vmin=0.75,vmax=4.75,levels=15,colors='black',linewidths=0.5)
    plt.title(name)

In [8]:
label='ShockEntropy'

for WENO,name in zip([
    WENO_Z_BI_sim,
    WENO_Z_sim,
    WENO_Zp_sim,
    WENO_Z_MS_sim,
    WENO_Zp_MS_sim,
    WENO_Zp_BI_sim,
    WENO_JS_BI_sim,
    WENO_ZC_MS_sim,
    WENO_ZC_BI_sim
    ],
    [
    'WENO-Z (BI)',
    'WENO-Z',
    'WENO-Z+',
    'WENO-Z (MS)',
    'WENO-Z+ (MS)',
    'WENO-Z+ (BI)',
    'WENO-JS (BI)',
    'WENO-ZC (MS)',
    'WENO-ZC (BI)'
    ]):
    frames=[]
    if not(os.path.isdir(f'imagens/{label}-{N}-{name}/')):
        os.mkdir(f'imagens/{label}-{N}-{name}/')
    if os.path.isfile(f'imagens/{label}-{N}-{name}/data.bkp'):
        with open(f'imagens/{label}-{N}-{name}/data.bkp','rb') as file:
            U_total=list(API_Numpy.unstack(dill.load(file),axis=0))
        U=U_total[-1].copy()
    else:
        U=U0.copy()
        U_total=[U0.copy()]
    count=len(U_total)-1
    Δt_max=0.001
    t=count*Δt_max
    print(name)
    print(f'Tempo inicial: {t}')
    while t<t_final:
        
        t_step=0
        while t_step<Δt_max:
            
            Λ  = WENO.equation.maximum_speed(U)
            Δt = min(Δx,Δy)*cfl/Λ
            Δt = min(Δt_max,Δt)
            Δt = np.where(t_step+Δt>Δt_max,Δt_max-t_step,Δt)

            U=WENO.Sim_step(U, Δt, Δx, Δy, GhostPointsX, GhostPointsY,NullForce, t=t+t_step+Δt)
            t_step+=Δt
        t+=Δt_max
        count+=1
        U_total.append(U)

        # helper_plot(U[0],x,y)
        # plt.savefig(f'imagens/{label}-{N}-{name}/{str(count).zfill(4)}.png')
        # plt.close()

        # frames.append(helper_gif(U[0],x,y))

        with open(f'imagens/{label}-{N}-{name}/data.bkp','wb') as file:
            dill.dump(np.stack(U_total,axis=0),file)

        print(f'Tempo atual: {t}                     ',end='\r')
    print(f'Tempo final: {t}')

    # helper_plot(U[0],x,y)
    # plt.savefig(f'imagens/{label}-{N}-{name}/{name}.png')
    # plt.close()

    # gif.save(frames,
    # f"imagens/{label}-{N}-{name}/{name}.gif", 
    #         duration=1)

WENO-ZC (MS)
Tempo inicial: 0.13
Tempo final: 0.20000000000000007                     
WENO-ZC (BI)
Tempo inicial: 0.0
Tempo final: 0.20000000000000015                      


In [10]:
t=0
count=0
for WENO,name in zip([
    WENO_Z_BI_sim,
    WENO_Z_sim,
    WENO_Zp_sim,
    WENO_Z_MS_sim,
    WENO_Zp_MS_sim,
    WENO_JS_BI_sim,
    WENO_ZC_MS_sim,
    WENO_ZC_BI_sim
    ],
    [
    'WENO-Z (BI)',
    'WENO-Z',
    'WENO-Z+',
    'WENO-Z (MS)',
    'WENO-Z+ (MS)',
    'WENO-JS (BI)',
    'WENO-ZC (MS)',
    'WENO-ZC (BI)'
    ]):

    frames=[]

    with open(f'imagens/{label}-{N}-{name}/data.bkp','rb') as file:
        U_total=dill.load(file)
    if not(os.path.isdir(f'imagens/{label}-{N}-{name}/')):
            os.mkdir(f'imagens/{label}-{N}-{name}/')
    for count,U in enumerate(API_Numpy.unstack(U_total,axis=0)):

        helper_plot(U[0],x,y)
        plt.savefig(f'imagens/{label}-{N}-{name}/{str(count).zfill(4)}.png')
        plt.close()

        frames.append(helper_gif(U[0],x,y))

        print(count,'                             ',end='\r')
    gif.save(frames,
            f"imagens/{label}-{N}-{name}/{name}.gif", 
            duration=1)